In [1]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    return run.split('_')[0]

dirs = [
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge/join_ssd'
]
runs = []
for dir in dirs:
 for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results')):
        if (run_name=="run"):
           continue
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results',  run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)
display(df)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'iostat.bytes_read', 'iostat.bytes_wrtn',
       'result.index_query_usec', 'result.last_mile_fetch_usec',
       'result.last_mile_search_usec', 'spec.index.epsilon',
       'spec.index.leaf_size_in_pages', 'spec.index.search',
       'spec.index.type'],
      dtype='object')


,command,dataset,result.checksum,result.duration_ns,result.duration_sec,result.inner_disk_fetch,result.inner_disk_fetch_size,result.inner_total_bytes_fetched,result.num_output_keys,result.outer_disk_fetch,...,spec.write_result_to_disk,iostat.bytes_read,iostat.bytes_wrtn,result.index_query_usec,result.last_mile_fetch_usec,result.last_mile_search_usec,spec.index.epsilon,spec.index.leaf_size_in_pages,spec.index.search,spec.index.type
0,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,72701457649,72.701458,781250,4096,3200000000,1990119,0,...,True,3162004000,32508000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,33244318358,33.244318,781250,4096,3200000000,1990119,7774,...,True,3162004000,31784000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,104756867814,104.756868,781250,4096,3200000000,199999999,781250,...,True,6256112000,3126736000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,135577641391,135.577641,781250,4096,3200000000,199999999,0,...,True,6256520000,3127240000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,72533067818,72.533068,781250,4096,3200000000,1990119,0,...,True,3162460000,32132000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,158791065201,158.791065,390627,8192,3200012288,199999741,781250,...,True,6265548000,3127620000,32755877.0,19658247.0,253979.0,256.0,1.0,binary,btree
68,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,58110797774,58.110798,374984,8192,3071868928,1990176,7775,...,True,3045948000,32932000,1379514.0,55006213.0,37282.0,256.0,1.0,binary,btree
69,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,62164455572,62.164456,374984,8192,3071868928,1990176,7775,...,True,3045324000,32064000,1539904.0,58389786.0,104788.0,256.0,1.0,binary,btree
70,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,35255914172,35.255914,388776,8192,3184852992,1990176,7775,...,True,3146472000,31752000,1060397.0,32533775.0,32202.0,256.0,NaN,binary,sampledflatpgm256


In [2]:

epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    return "NA"

df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["inner_disk_fetch"] = df["result.inner_disk_fetch"]
df['result_num_keys'] = df['result.num_output_keys']
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["epsilon"] = df["spec.index.epsilon"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['disk_read_gb'] = df['iostat.bytes_read']/1e9
df['disk_wrtn_gb'] = df['iostat.bytes_wrtn']/1e9
#df['cdisk_read_gb'] = df['cgroup_iostat.bytes_read']/1e9
#df['cdisk_wrtn_gb'] = df['cgroup_iostat.bytes_wrtn']/1e9
df['checksum'] = df['result.checksum']
df['threads'] = df['spec.num_threads']

df['dataset'].unique()
df['result.num_output_keys'].unique()
duckdb.sql("SELECT * FROM df WHERE dataset='fb'")


┌──────────────────────┬─────────┬─────────────────┬───┬────────────┬──────────────┬──────────────┬──────────┐
│       command        │ dataset │ result.checksum │ … │ index_type │ disk_read_gb │ disk_wrtn_gb │ checksum │
│       varchar        │ varchar │      int64      │   │  varchar   │    double    │    double    │  int64   │
├──────────────────────┼─────────┼─────────────────┼───┼────────────┼──────────────┼──────────────┼──────────┤
│ sponge/join_ssd/bu…  │ fb      │               0 │ … │ NA         │     3.162004 │     0.032508 │        0 │
│ sponge/join_ssd/bu…  │ fb      │               0 │ … │ NA         │     3.162004 │     0.031784 │        0 │
│ sponge/join_ssd/bu…  │ fb      │               0 │ … │ NA         │     6.256112 │     3.126736 │        0 │
│ sponge/join_ssd/bu…  │ fb      │               0 │ … │ NA         │      6.25652 │      3.12724 │        0 │
│ sponge/join_ssd/bu…  │ fb      │               0 │ … │ NA         │      3.16246 │     0.032132 │        0 │
│

In [4]:
def print_for_dataset(dataset, algos):
    print("## " + dataset)
    algo_filter = ''
    for algo in algos:
        algo_filter = algo_filter + f" OR algo='{algo}'"
    algo_filter = algo_filter[3:]
    rows = duckdb.sql(
        " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    print(rows[algos].to_markdown())
    rows[algos].to_csv('/home/chesetti/Repos/LearnedJoinDiskExp/experiments/results/join_ssd/'+dataset+'.txt')

    print("internal counter GB read")
    rows = duckdb.sql(
        " SELECT ratio, algo, MIN(inner_bytes_fetched) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    print(rows[algos].to_markdown())
    print('iostat GB read')
    rows = duckdb.sql(
        " SELECT ratio, algo, MIN(disk_read_gb) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    print(rows[algos].to_markdown())

    print("internal counter GB read")
    rows = duckdb.sql(
        " SELECT ratio, algo, MIN(inner_bytes_fetched) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    print(rows[algos].to_markdown())

    print("num output keys")
    rows = duckdb.sql(
        " SELECT ratio, algo, MIN(result_num_keys) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    print(rows[algos].to_markdown())


algos = ['lsj_sampledflatpgm256', 'inlj_sampledflatpgm256', 'lsj_btree256', 'inlj_btree256',  'sort_join', 'hash_join']
print_for_dataset("fb", algos)
#print_for_dataset("wiki", algos)
#print_for_dataset("osm", algos)
#print_for_dataset("books", algos)



## fb
|   ratio |   lsj_sampledflatpgm256 |   inlj_sampledflatpgm256 |   lsj_btree256 |   inlj_btree256 |   sort_join |   hash_join |
|--------:|------------------------:|-------------------------:|---------------:|----------------:|------------:|------------:|
|       1 |                130.295  |               14664.1    |       142.771  |        158.791  |    105.097  |    135.302  |
|      10 |                 31.967  |                1494.76   |        33.2318 |         34.6063 |     43.1566 |     84.0768 |
|     100 |                 35.2559 |                 262.132  |        58.1108 |         57.8933 |     34.3775 |     72.5331 |
|    1000 |                 35.7809 |                  40.2116 |        33.1366 |         34.6245 |     33.7642 |     46.8658 |
internal counter GB read
|   ratio |   lsj_sampledflatpgm256 |   inlj_sampledflatpgm256 |   lsj_btree256 |   inlj_btree256 |   sort_join |   hash_join |
|--------:|------------------------:|-------------------------:|---------

In [5]:
rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='fb'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='fb'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='1024') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='wiki'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='4096') AND dataset='books'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='osm'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))



algo,hash_join,inlj_btree256,inlj_sampledflatpgm256,lsj_btree256,lsj_sampledflatpgm256,sort_join
ratio,,,,,,
1,135.301928,158.791065,14664.057165,142.771412,130.294550,105.097146
10,84.076842,34.606260,1494.757512,33.231845,31.966999,43.156629
100,72.533068,57.893338,262.132193,58.110798,35.255914,34.377485
1000,46.865834,34.624467,40.211589,33.136582,35.780863,33.764243


algo,lsj_sampledflatpgm256
ratio,
1,130.294550
10,31.966999
100,35.255914
1000,35.780863


algo
ratio


algo
ratio


algo
ratio


algo
ratio


In [6]:
rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='1024') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='4096') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))


algo
ratio


algo
ratio


algo
ratio


In [8]:
def print_for_dataset(dataset, algos):
    print("## " + dataset)
    algo_filter = ''
    for algo in algos:
        algo_filter = algo_filter + f" OR algo='{algo}'"
    algo_filter = algo_filter[3:]
    rows = duckdb.sql(
        " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    print(rows[algos].to_markdown())
    rows[algos].to_csv('all_indexes/'+dataset+'.csv')



algos = ['lsj_sampledflatpgm256', 'lsj_btree256',  'sort_join', 'hash_join']
print_for_dataset("fb", algos)
#print_for_dataset("wiki", algos)
#print_for_dataset("osm", algos)
#print_for_dataset("books", algos)



## fb
|   ratio |   lsj_sampledflatpgm256 |   lsj_btree256 |   sort_join |   hash_join |
|--------:|------------------------:|---------------:|------------:|------------:|
|       1 |                130.295  |       142.771  |    105.097  |    135.302  |
|      10 |                 31.967  |        33.2318 |     43.1566 |     84.0768 |
|     100 |                 35.2559 |        58.1108 |     34.3775 |     72.5331 |
|    1000 |                 35.7809 |        33.1366 |     33.7642 |     46.8658 |


OSError: Cannot save file into a non-existent directory: 'all_indexes'

In [34]:
algos = ['lsj_sampledflatpgm256', 'lsj_flatpgm256', 'lsj_radixspline256', 'lsj_rmi' , 'lsj_btree256', 'sort_join']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)


## fb
|   ratio |   lsj_sampledflatpgm256 |   lsj_flatpgm256 |   lsj_radixspline256 |   lsj_rmi |   lsj_btree256 |   sort_join |
|--------:|------------------------:|-----------------:|---------------------:|----------:|---------------:|------------:|
|       1 |               23.2294   |        22.4319   |            25.6702   | 20.3287   |       42.3009  |    11.7042  |
|      10 |                3.68178  |         3.45073  |             4.30738  |  3.76312  |        5.83421 |     4.45029 |
|     100 |                0.98362  |         1.01192  |             1.46279  |  1.25237  |        1.57196 |     3.12833 |
|    1000 |                0.385151 |         0.353513 |             0.374326 |  0.275802 |        0.31908 |     2.86744 |
## wiki
|   ratio |   lsj_sampledflatpgm256 |   lsj_flatpgm256 |   lsj_radixspline256 |   lsj_btree256 |   sort_join |
|--------:|------------------------:|-----------------:|---------------------:|---------------:|------------:|
|       1 |               

In [29]:
algos = ['lsj_radixspline256', 'lsj_radixspline1024', 'lsj_radixspline4096']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)

## fb


BinderException: Binder Error: Referenced column "threads" not found in FROM clause!
Candidate bindings: "df.dataset"